# 6.1 读写文本格式的数据

pandas提供了一些用于将表格型数据读取为DataFrame对象的函数。表6-1对它们进行了总结，其中read_csv和read_table可能会是你今后用得最多的。

日期和其他自定义类型的处理需要多花点工夫才行。首先我们来看一个以逗号分隔的（CSV）文本文件：

由于该文件以逗号分隔，所以我们可以使用read_csv将其读入一个DataFrame：

In [1]:
import pandas as pd
import numpy as np


csv_path = r'../../data_examples/read_files'

In [2]:
# df = pd.read_csv('%s/ex1.csv' % csv_path)


file_v1 = r'%s/ex1.csv' % csv_path


df = pd.read_csv(file_v1)

In [3]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


我们还可以使用read_table，并指定分隔符：

In [4]:
pd.read_table(file_v1, sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


并不是所有文件都有标题行。看看下面这个文件：
读入该文件的办法有两个。你可以让pandas为其分配默认的列名，也可以自己定义列名：

In [5]:
file_v2 = r'%s/ex2.csv' % csv_path

pd.read_csv(file_v2, header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_csv(file_v2, names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


假设你希望将message列做成DataFrame的索引。你可以明确表示要将该列放到索引4的位置上，也可以通过index_col参数指定"message"：

In [7]:
names = ['a', 'b', 'c', 'd', 'message']

In [8]:
pd.read_csv(file_v2, names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [9]:
csv_mindex = r'%s/csv_mindex.csv' % csv_path


parsed = pd.read_csv(csv_mindex,
                     index_col=['key1', 'key2'])

如果希望将多个列做成一个层次化索引，只需传入由列编号或列名组成的列表即可：

In [10]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

有些情况下，有些表格可能不是用固定的分隔符去分隔字段的（比如空白符或其它模式）。看看下面这个文本文件：

In [11]:
ex3Txt = r'%s/ex3.txt' % csv_path

list(open(ex3Txt))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

虽然可以手动对数据进行规整，这里的字段是被数量不同的空白字符间隔开的。这种情况下，你可以传递一个正则表达式作为read_table的分隔符。可以用正则表达式表达为\s+，于是有：

In [12]:
result = pd.read_table(ex3Txt, sep='\s+')

result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


这里，由于列名比数据行的数量少，所以read_table推断第一列应该是DataFrame的索引。

这些解析器函数还有许多参数可以帮助你处理各种各样的异形文件格式（表6-2列出了一些）。比如说，你可以用skiprows跳过文件的第一行、第三行和第四行：

In [14]:
ex4Csv = r'%s/ex4.csv' % csv_path

pd.read_csv(ex4Csv, skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


缺失值处理是文件解析任务中的一个重要组成部分。缺失数据经常是要么没有（空字符串），要么用某个标记值表示。默认情况下，pandas会用一组经常出现的标记值进行识别，比如NA及NULL：

In [16]:
ex5Csv = r'%s/ex5.csv' % csv_path

result = pd.read_csv(ex5Csv)

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [17]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


na_values可以用一个列表或集合的字符串表示缺失值：

In [18]:
result = pd.read_csv(ex5Csv, na_values=['NULL'])

result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


字典的各列可以使用不同的NA标记值：

In [19]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}


pd.read_csv(ex5Csv, na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 逐块读取文本文件

在处理很大的文件时，或找出大文件中的参数集以便于后续处理时，你可能只想读取文件的一小部分或逐块对文件进行迭代。

在看大文件之前，我们先设置pandas显示地更紧些：

In [25]:
pd.options.display.max_rows = 10

In [26]:
ex6Csv = r'%s/ex6.csv' % csv_path

result = pd.read_csv(ex6Csv)

result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


如果只想读取几行（避免读取整个文件），通过nrows进行指定即可：

In [35]:
pd.read_csv(ex6Csv, nrows=10)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


要逐块读取文件，可以指定chunksize（行数）：

In [47]:
print(ex6Csv)


import time
start = time.perf_counter() # calculate running time
chunksize = 10000


reader = pd.read_csv(ex6Csv,
                     encoding='utf-8',
                     on_bad_lines='skip',
                     chunksize=chunksize,
                     iterator=True)


# 'on_bad_lines' is to solve: Error tokenizing data. C error: Expected 1 fields in line 3, saw 2
for i, ck in enumerate(reader):
    print(i, ck)
    pass


end = time.perf_counter()
print('using time: %s seconds' % (end-start))

../../data_examples/read_files/ex6.csv
0            one       two     three      four key
0     0.467976 -0.038649 -0.295344 -1.824726   L
1    -0.358893  1.404453  0.704965 -0.200638   B
2    -0.501840  0.659254 -0.421691 -0.057688   G
3     0.204886  1.074134  1.388361 -0.982404   R
4     0.354628 -0.133116  0.283763 -0.837063   Q
...        ...       ...       ...       ...  ..
9995  2.311896 -0.417070 -1.409599 -0.515821   L
9996 -0.479893 -0.650419  0.745152 -0.646038   E
9997  0.523331  0.787112  0.486066  1.093156   K
9998 -0.362559  0.598894 -1.843201  0.887292   G
9999 -0.096376 -1.012999 -0.657431 -0.573315   0

[10000 rows x 5 columns]
using time: 0.005747547984356061 seconds


In [52]:
tot = pd.Series([])


for piece in reader:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)


tot = tot.sort_values(ascending=False)

tot

/tmp/ipykernel_11977/1698026734.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = pd.Series([])


Series([], dtype: float64)

## 将数据写出到文本格式

数据也可以被输出为分隔符格式的文本。我们再来看看之前读过的一个CSV文件：

In [55]:
ex5Csv = r'%s/ex5.csv' % csv_path

data = pd.read_csv(ex5Csv)

data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


利用DataFrame的to_csv方法，我们可以将数据写到一个以逗号分隔的文件中：

In [56]:
data.to_csv('./out_5.csv')

当然，还可以使用其他分隔符（由于这里直接写出到sys.stdout，所以仅仅是打印出文本结果而已）：

In [57]:
import sys

data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


缺失值在输出结果中会被表示为空字符串。你可能希望将其表示为别的标记值：

In [58]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


如果没有设置其他选项，则会写出行和列的标签。当然，它们也都可以被禁用：

In [59]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


此外，你还可以只写出一部分的列，并以你指定的顺序排列：

In [61]:
data.to_csv(sys.stdout, 
            index=False, 
            columns=['a', 'b', 'c']
           )

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series也有一个to_csv方法：

In [62]:
dates = pd.date_range('1/1/2000', periods=7)

dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
ts = pd.Series(np.arange(7), index=dates)

print(ts)

ts.to_csv('./tseries.csv')

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int64


## 处理分隔符格式

大部分存储在磁盘上的表格型数据都能用pandas.read_table进行加载。然而，有时还是需要做一些手工处理。由于接收到含有畸形行的文件而使read_table出毛病的情况并不少见。为了说明这些基本工具，看看下面这个简单的CSV文件：

对于任何单字符分隔符文件，可以直接使用Python内置的csv模块。将任意已打开的文件或文件型的对象传给csv.reader：

In [77]:
import csv


ex7Csv = r'%s/ex7.csv' % csv_path
f = open(ex7Csv)

reader = csv.reader(f)

reader

对这个reader进行迭代将会为每行产生一个元组（并移除了所有的引号）：对这个reader进行迭代将会为每行产生一个元组（并移除了所有的引号）：

In [68]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


现在，为了使数据格式合乎要求，你需要对其做一些整理工作。我们一步一步来做。首先，读取文件到一个多行的列表中：

In [70]:
with open(ex7Csv) as f:
    lines = list(csv.reader(f))


# 然后，我们将这些行分为标题行和数据行：
header, values = lines[0], lines[1:]

# 然后，我们可以用字典构造式和zip(*values)，后者将行转置为列，创建数据列的字典：
data_dict = {h: v for h, v in zip(header, zip(*values))}

data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV文件的形式有很多。只需定义csv.Dialect的一个子类即可定义出新格式（如专门的分隔符、字符串引用约定、行结束符等）：

class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL


    
reader = csv.reader(f, dialect=my_dialect)

reader

各个CSV语支的参数也可以用关键字的形式提供给csv.reader，而无需定义子类：

In [79]:
reader = csv.reader(f, delimiter='|')

reader

可用的选项（csv.Dialect的属性）及其功能如表6-3所示。

要手工输出分隔符文件，你可以使用csv.writer。它接受一个已打开且可写的文件对象以及跟csv.reader相同的那些语支和格式化选项：

In [82]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))